In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import itertools
import json
import sys
# sys.path.append("..")
from utils import data_proc_tools as dpt
from utils import plot_tools as pt
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.custom_metrics import recall_np, precision_np, binary_accuracy_np, multilabel_confusion_matrix
from utils.rnn_textsum_models import HierSeq2SeqAtt
import random
random.seed(42)
random_state=1000
pd.set_option('display.max_colwidth', -1)
import pylab

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

Using TensorFlow backend.
W1210 14:06:33.595479 140602779563776 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1210 14:06:33.808887 140602779563776 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1210 14:06:33.959377 140602779563776 __init__.py:352] Limited tf.summary API due to missing TensorBoard installation.
W1210 14:06:34.411010 140602779563776 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


In [3]:
dir = '/vol/medic02/users/ag6516/radiology_report_summarisation/'
data_dir = dir + 'data/'

aug = 'aug'

model_output_dir = dir + 'trained_models/hierseq2seq/'

train_df = pd.read_pickle(data_dir + 'train/{}_train.pkl'.format(aug))
val_df = pd.read_pickle(data_dir + 'val/val.pkl')

## Load and prepare sequence data for training

In [4]:
train_df.head()

,examid,report,all_mesh,single_mesh
0,CXR1000_IM-0003,"[increased, opacity, within, right, upper, lobe, possible, mass, associated, area, atelectasis, focal, consolidation, ., cardiac, silhouette, within, normal, limits, ., opacity, left, midlung, overlying, posterior, left, 5th, rib, may, represent, focal, airspace, disease, ., increased, opacity, right, upper, lobe, associated, atelectasis, may, represent, focal, consolidation, mass, lesion, atelectasis, ., recommend, chest, ct, evaluation, ., opacity, overlying, left, 5th, rib, may, represent, focal, airspace, disease]","[opacity, lung, lingula, opacity, lung, upper_lobe, right, pulmonary_atelectasis, upper_lobe, right]","[opacity, lung, upper_lobe, right]"
1,CXR1001_IM-0004,"[interstitial, markings, diffusely, prominent, throughout, lungs, ., heart, size, normal, ., pulmonary, normal, ., diffuse, fibrosis]","[diffuse, markings, lung, bilateral, interstitial, diffuse, prominent]","[markings, lung, bilateral, interstitial, diffuse, prominent]"
2,CXR1002_IM-0004,"[status, post, left, mastectomy, ., heart, size, normal, ., lungs, clear]",[left],[left]
3,CXR1003_IM-0005,"[heart, size, pulmonary, vascularity, appear, within, normal, limits, ., retrocardiac, soft, tissue, density, present, ., appears, air, within, suggest, represents, hiatal, hernia, ., vascular, calcification, noted, ., calcified, granuloma, seen, ., interval, development, bandlike, opacity, left, lung, base, ., may, represent, atelectasis, ., osteopenia, present, spine, ., retrocardiac, soft, tissue, density, ., appearance, suggests, hiatal, hernia, ., left, base, bandlike, opacity, ., appearance, suggests, atelectasis]","[bone_diseases_metabolic, spine, calcified_granuloma, calcinosis, blood_vessels, density, retrocardiac, opacity, lung, base, left]","[opacity, lung, base, left]"
4,CXR1004_IM-0005,"[heart, ,, pulmonary, mediastinum, within, normal, limits, ., aorta, tortuous, ectatic, ., degenerative, changes, acromioclavicular, joints, ., degenerative, changes, spine, ., ivc, identified]","[aorta, tortuous, catheters_indwelling, shoulder, bilateral, degenerative, spine, degenerative]","[shoulder, bilateral, degenerative]"


In [5]:
# prepend and append start and end tokens to mesh captions and text reports
start_token = 'start'
end_token = 'end'
unknown_token = '**unknown**'
max_mesh_length = 13 # avg. + 1std. + start + end
max_num_words = 11 # avg. + 1std. + start + end, max number of words per sentence
max_num_sentences = 6 # avg. + 1std. + start + end, max number of sentences per report

In [6]:
def split_pad_report(report, max_num_sentences, max_num_words, start_token, end_token):
    pad_seq = [end_token for x in range(max_num_words)]
    r = ' '.join(report)
    sentences = r.split(' . ')
    padded_sentences = []
    for sen in sentences:
        words = sen.split(' ')
        padded_sentence = dpt.pad_sequence(words, max_num_words, start_token, end_token)
        padded_sentences.append(padded_sentence)
    if len(padded_sentences) >= max_num_sentences:
        padded_sentences = padded_sentences[:max_num_sentences]
    else:
        while len(padded_sentences) < max_num_sentences:
            padded_sentences.append(pad_seq)
    return padded_sentences

In [7]:
train_df['pad_mesh_caption'] = train_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
train_df['pad_text_report'] = train_df.report.apply(lambda x: split_pad_report(x, max_num_sentences, max_num_words, start_token, end_token))

val_df['pad_mesh_caption'] = val_df.all_mesh.apply(lambda x: dpt.pad_sequence(x, max_mesh_length, start_token, end_token))
val_df['pad_text_report'] = val_df.report.apply(lambda x: split_pad_report(x, max_num_sentences, max_num_words, start_token, end_token))

## Vectorise text reports and mesh captions

In [8]:
train_mesh = list(train_df.pad_mesh_caption)
train_reports = list(train_df.pad_text_report)

# vectorize train mesh captions
dpt.mesh_to_vectors(train_mesh, dicts_dir=data_dir+'dicts/', 
                    load_dicts=True, save=True, 
                    output_dir=data_dir+'train/')

# vectorise train reports
vec_reports = []
for report in train_reports:
    vec = dpt.reports_to_vectors(report, 
                                 dicts_dir=data_dir+'dicts/', 
                                 load_dicts=True, 
                                 output_dir=data_dir+'train/')
    vec_reports.append(vec)
vec_reports = np.array(vec_reports)
np.save(data_dir+'train/' + 'sent_token_ids_array.npy', vec_reports)

In [9]:
val_reports = list(val_df.pad_text_report)
val_mesh = list(val_df.pad_mesh_caption)

# vectorise val mesh using the same dict as created for train
dpt.mesh_to_vectors(val_mesh, dicts_dir=data_dir+'dicts/', 
                    load_dicts=True, save=True, 
                    output_dir=data_dir+'val/')

# vectorise val reports
vec_reports = []
for report in val_reports:
    vec = dpt.reports_to_vectors(report, 
                                 dicts_dir=data_dir+'dicts/', 
                                 load_dicts=True, 
                                 output_dir=data_dir+'train/')
    vec_reports.append(vec)
vec_reports = np.array(vec_reports)
np.save(data_dir+'val/' + 'sent_token_ids_array.npy', vec_reports)

In [10]:
word_to_id, id_to_word = dpt.load_report_dicts(data_dir+'dicts/')
mesh_to_id, id_to_mesh = dpt.load_mesh_dicts(data_dir+'dicts/')

report_vocab_length = len(word_to_id)
mesh_vocab_length = len(mesh_to_id)

In [11]:
report_vocab_length, mesh_vocab_length

(1475, 128)

In [12]:
# Create arrays of indixes for input sentences, output entities and shifted output entities (t-1)
train_token_ids_array = np.load(data_dir + 'train/sent_token_ids_array.npy')
train_mesh_ids_array = np.load(data_dir + 'train/mesh_ids_array.npy')
train_mesh_ids_array_shifted =[np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in train_mesh_ids_array]
train_mesh_ids_array_shifted = np.asarray(train_mesh_ids_array_shifted)

val_token_ids_array = np.load(data_dir + 'val/sent_token_ids_array.npy')
val_mesh_ids_array = np.load(data_dir + 'val/mesh_ids_array.npy')
val_mesh_ids_array_shifted = [np.concatenate((mesh_to_id[start_token], t[:-1]), axis=None) for t in val_mesh_ids_array]
val_mesh_ids_array_shifted = np.asarray(val_mesh_ids_array_shifted)

In [14]:
# one-hot-encode
# one_hot_reports_train = []
# for report in train_token_ids_array:
#     one_hot_reports_train.append(dpt.one_hot_sequence(report, report_vocab_length))
# one_hot_reports_train = np.array(one_hot_reports_train)
#dpt.one_hot_sequence(train_token_ids_array, report_vocab_length)
one_hot_mesh_train = dpt.one_hot_sequence(train_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_train = dpt.one_hot_sequence(train_mesh_ids_array_shifted, mesh_vocab_length)

# one_hot_reports_val = []
# for report in val_token_ids_array:
#     one_hot_reports_val.append(dpt.one_hot_sequence(report, report_vocab_length))
# one_hot_reports_train = np.array(one_hot_reports_train)
one_hot_mesh_val = dpt.one_hot_sequence(val_mesh_ids_array, mesh_vocab_length)
one_hot_mesh_shifted_val = dpt.one_hot_sequence(val_mesh_ids_array_shifted, mesh_vocab_length)

In [15]:
one_hot_mesh_train.shape, one_hot_mesh_shifted_train.shape

((5148, 13, 128), (5148, 13, 128))

In [ ]:
train_mesh_ids_array[0]

## Train Seq-to-Seq Model

In [29]:
from utils.custom_metrics import recall, precision, binary_accuracy
from utils.rnn_textsum_models import HierSeq2SeqAtt

input_dim = len(word_to_id)
output_dim = len(mesh_to_id)
hidden_dim = 64
encoder_emb_dim = 128
input_word_seq_length = max_num_words
input_sent_seq_length = max_num_sentences
output_seq_length = max_mesh_length
epochs = 50
optimizer = 'adam'
loss='categorical_crossentropy'
batch_size = 128

new_experiment = HierSeq2SeqAtt(epochs=epochs,
                               metrics=['accuracy', binary_accuracy,recall,precision],
                               optimizer=optimizer,
                               loss=loss,
                               batch_size=batch_size, 
                               input_dim=input_dim,
                               output_dim=output_dim,
                               hidden_dim=hidden_dim,
                               encoder_emb_dim=encoder_emb_dim,
                               input_word_seq_length=input_word_seq_length,
                               input_sent_seq_length=input_sent_seq_length,
                               output_seq_length=output_seq_length,
                               verbose=True)
new_experiment.build_model()
new_experiment.model.summary()

Word encoder emb:  (None, 11, 128)
Sent emb:  (None, 64)
Encoder outputs:  (None, 6, 64)
Encoder state:  (None, 64)
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 6, 11)]      0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 6, 64)        238208      input_18[0][0]                   
__________________________________________________________________________________________________
input_19 (InputLayer)           [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, 6, 64), (Non 33024       time_dist

In [ ]:
# create batch generators
# train_batch_generator = dpt.batch_generator_seq2seq(train_token_ids_array, report_vocab_length, train_mesh_ids_array, 
#                                                    train_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

# val_batch_generator = dpt.batch_generator_seq2seq(val_token_ids_array, report_vocab_length, val_mesh_ids_array, 
#                                                    val_mesh_ids_array_shifted, mesh_vocab_length, batch_size)

In [30]:
new_experiment.run_experiment(train_token_ids_array, one_hot_mesh_shifted_train, one_hot_mesh_train, 
                              val_token_ids_array, one_hot_mesh_shifted_val, one_hot_mesh_val)

Word encoder emb:  (None, 11, 128)
Sent emb:  (None, 64)
Encoder outputs:  (None, 6, 64)
Encoder state:  (None, 64)
Train on 5148 samples, validate on 300 samples
Epoch 1/50
5148/5148 [==============================] - 9s 2ms/sample - loss: 2.9574 - accuracy: 0.5281 - binary_accuracy: 0.9935 - recall: 0.2852 - precision: 0.4583 - val_loss: 1.9513 - val_accuracy: 0.6526 - val_binary_accuracy: 0.9950 - val_recall: 0.5052 - val_precision: 0.7870
Epoch 2/50
5148/5148 [==============================] - 2s 437us/sample - loss: 2.0800 - accuracy: 0.5971 - binary_accuracy: 0.9951 - recall: 0.4937 - precision: 0.8144 - val_loss: 1.6771 - val_accuracy: 0.6574 - val_binary_accuracy: 0.9964 - val_recall: 0.5946 - val_precision: 0.9209
Epoch 3/50
5148/5148 [==============================] - 2s 438us/sample - loss: 1.8393 - accuracy: 0.6158 - binary_accuracy: 0.9961 - recall: 0.5378 - precision: 0.9372 - val_loss: 1.4854 - val_accuracy: 0.7031 - val_binary_accuracy: 0.9968 - val_recall: 0.6098 - val

5148/5148 [==============================] - 5s 934us/sample - loss: 0.5779 - accuracy: 0.8552 - binary_accuracy: 0.9980 - recall: 0.7719 - precision: 0.9631 - val_loss: 0.6780 - val_accuracy: 0.8372 - val_binary_accuracy: 0.9979 - val_recall: 0.7900 - val_precision: 0.9451
Epoch 30/50
5148/5148 [==============================] - 5s 929us/sample - loss: 0.5543 - accuracy: 0.8620 - binary_accuracy: 0.9981 - recall: 0.7809 - precision: 0.9652 - val_loss: 0.6653 - val_accuracy: 0.8413 - val_binary_accuracy: 0.9980 - val_recall: 0.7904 - val_precision: 0.9505
Epoch 31/50
5148/5148 [==============================] - 5s 936us/sample - loss: 0.5357 - accuracy: 0.8657 - binary_accuracy: 0.9981 - recall: 0.7864 - precision: 0.9653 - val_loss: 0.6673 - val_accuracy: 0.8397 - val_binary_accuracy: 0.9980 - val_recall: 0.7924 - val_precision: 0.9456
Epoch 32/50
5148/5148 [==============================] - 5s 918us/sample - loss: 0.5242 - accuracy: 0.8682 - binary_accuracy: 0.9981 - recall: 0.7915 -

In [31]:
model_name = 'hierseq2seq_att'
model_output_dir = dir + 'trained_models/{}/'.format(model_name)
new_experiment.save_weights_history(model_output_dir)

## Load results of specific experiment

In [16]:
model_name = 'hierseq2seq_att'
model_output_dir = dir + 'trained_models/{}/'.format(model_name)

In [32]:
import pickle

model_name = 'hierseq2seq'
epochs = 50
encoder_emb_dim = 128
decoder_emb_dim = 256
hidden_dim = 64

param_fn = 'param_encoderembdim_{}_decoderembdim_{}_hiddendim_{}_epochs_{}.pkl'.format(encoder_emb_dim,
                                                                               decoder_emb_dim,
                                                                               hidden_dim,
                                                                               epochs)
params = pickle.load(open(model_output_dir + param_fn, 'rb'))

old_experiment = HierSeq2SeqAtt(**params)
old_experiment.build_model()
old_experiment.load_weights_history(model_output_dir)

Word encoder emb:  (None, 11, 128)
Sent emb:  (None, 64)
Encoder outputs:  (None, 6, 64)
Encoder state:  (None, 64)


In [33]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [34]:
def strip_start_end(seq, start_token='start', end_token='end'):
    stripped_seq = []
    for s in seq:
        if s not in [start_token, end_token]:
            stripped_seq.append(s)
    return stripped_seq

In [35]:
def predict_sequence(experiment, source, max_seq_len, id_to_mesh, start_token='start', end_token='end'):
    # encode source
    enc_outs, h, c = experiment.encoder_model.predict(source)
    enc_state = [h,c]
    dec_state = enc_state

    # start of sequence input
    in_text = [start_token]

    # integer encoder
    in_seq_ids = dpt.mesh_to_vectors([in_text], dicts_dir=data_dir+'dicts/', 
                   load_dicts=True, save=False)
    # one-hot encode
    in_seq_onehot = dpt.one_hot_sequence(in_seq_ids, mesh_vocab_length)
    in_seq_onehot = np.array(in_seq_onehot)
    in_seq_onehot = in_seq_onehot.reshape(1, 1, in_seq_onehot.shape[-1])
    target_seq = in_seq_onehot
    
    # collect predictions
    output = []
    attention_weights = []
    max_att = []
    max_att2 = []
    for t in range(max_seq_len):
        dec_out, attention, h, c  = experiment.decoder_model.predict([enc_outs] + dec_state + [target_seq])
        dec_state = [h,c]
        # store prediction
        #output.append(dec_out[0,0,:])
        dec_ind = np.argmax(dec_out, axis=-1)[0, 0]
        output.append(dec_ind)
        #print(dec_ind)
        attention_weights.append((dec_ind, attention))
        idx = np.argmax(attention, axis=-1)[0, 0]
        max_att.append(np.argmax(attention, axis=-1)[0, 0])
        attention[:,:,idx] = 0
        max_att2.append(np.argmax(attention, axis=-1)[0, 0])
        target_seq = dec_out

    #predicted_mesh_ids = one_hot_decode(output)
    predicted_mesh = [id_to_mesh[i] for i in output]
    
    return predicted_mesh, attention_weights, max_att, max_att2

In [36]:
for _ in range(1):
    sample = val_df.sample(1, random_state=42)
    true_mesh_caption = list(sample.all_mesh)[0]
    sample_report = list(sample.pad_text_report)[0]
    
    sample_report_ids = []
    for sent in sample_report:
        sent_ids = []
        for token in sent:
            if token in word_to_id.keys():
                sent_ids.append(word_to_id[token])
            else:
                sent_ids.append(word_to_id[unknown_token])
        sample_report_ids.append(sent_ids)

    sample_report_ids = np.array(sample_report_ids)
    sample_report_ids = sample_report_ids.reshape(1, sample_report_ids.shape[0], sample_report_ids.shape[1])
    #print(sample_report_ids.shape)
#     one_hot_sample_report = dpt.one_hot_sequence(sample_report_ids, report_vocab_length)
#     one_hot_sample_report = one_hot_sample_report[np.newaxis,:,:,:]
    #print(one_hot_sample_report.shape)
    prediction, attention_weights, max_att1, max_att2 = predict_sequence(old_experiment,
                                  sample_report_ids, 
                                  max_mesh_length, 
                                  id_to_mesh,
                                  start_token=start_token,
                                  end_token=end_token)
    #predicted_mesh_ids = one_hot_decode(prediction)
    #predicted_mesh = [id_to_mesh[idx] for idx in predicted_mesh_ids]
    
    sample_report = strip_start_end(sample_report)
    predicted_mesh = strip_start_end(prediction)
    
    print('')
    print('Original report: ', sample_report)
    print('True mesh caption: ', true_mesh_caption)
    print('Predicted mesh caption: ', predicted_mesh)
    
    att_words1 = [sample_report[k] for k in max_att1]
    att_words2 = [sample_report[k] for k in max_att2]

    print('Attention word inputs 1: ', att_words1)
    print('Attention word inputs 2: ', att_words2)


Original report:  [['start', 'interval', 'cabg', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end'], ['start', 'sternotomy', 'appear', 'intact', 'end', 'end', 'end', 'end', 'end', 'end', 'end'], ['start', 'stable', ',', 'mild', 'degenerative', 'disc', 'disease', 'thoracic', 'spine', 'end', 'end'], ['start', 'visualized', 'bony', 'structures', 'otherwise', 'unremarkable', 'appearance', 'end', 'end', 'end', 'end'], ['start', 'atherosclerotic', 'calcifications', 'thoracic', 'aorta', 'end', 'end', 'end', 'end', 'end', 'end'], ['start', 'clear', 'lungs', 'end', 'end', 'end', 'end', 'end', 'end', 'end', 'end']]
True mesh caption:  ['atherosclerosis', 'aorta_thoracic', 'thoracic_vertebrae', 'degenerative', 'mild']
Predicted mesh caption:  ['aorta_thoracic', 'tortuous', 'mild', 'thoracic_vertebrae', 'thoracic_vertebrae', 'degenerative', 'mild']
Attention word inputs 1:  [['start', 'sternotomy', 'appear', 'intact', 'end', 'end', 'end', 'end', 'end', 'end', 'end'], ['start', 'atherosclerot

## Evaluate BLEU scores on all trian/val/test data

In [37]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def evaluate_model(model, df, report_vocab_length):
    actual, predicted = list(), list()
    bleu1, bleu2, bleu3, bleu4 = list(), list(), list(), list()

    for _, sample in df.iterrows():
        true_mesh_caption = sample.all_mesh
        sample_report = sample.pad_text_report
        
        sample_report_ids = []
        for sent in sample_report:
            sent_ids = []
            for token in sent:
                if token in word_to_id.keys():
                    sent_ids.append(word_to_id[token])
                else:
                    sent_ids.append(word_to_id[unknown_token])
            sample_report_ids.append(sent_ids)

        sample_report_ids = np.array(sample_report_ids)
        sample_report_ids = sample_report_ids.reshape(1, sample_report_ids.shape[0], sample_report_ids.shape[1])
        prediction, _, _, _= predict_sequence(old_experiment,
                                      sample_report_ids, 
                                      max_mesh_length, 
                                      id_to_mesh,
                                      start_token=start_token,
                                      end_token=end_token)

        yhat = strip_start_end(prediction)
        reference = true_mesh_caption
        
        # calculate BLEU score
        bleu1.append(sentence_bleu([reference], yhat, weights=(1.0, 0, 0, 0)))
        bleu2.append(sentence_bleu([reference], yhat, weights=(0.5, 0.5, 0, 0)))
        bleu3.append(sentence_bleu([reference], yhat, weights=(0.3, 0.3, 0.3, 0)))
        bleu4.append(sentence_bleu([reference], yhat, weights=(0.25, 0.25, 0.25, 0.25)))
    
        # store actual and predicted
        actual.append(reference)
        predicted.append(yhat)
        
    print('BLEU1: ', np.mean(bleu1)*100)
    print('BLEU2: ', np.mean(bleu2)*100)
    print('BLEU3: ', np.mean(bleu3)*100)
    print('BLEU4: ', np.mean(bleu4)*100)
    
    return actual, predicted

In [42]:
train_actual, train_predicted = evaluate_model(old_experiment, train_df.sample(2000), report_vocab_length)

BLEU1:  75.61706725973593
BLEU2:  45.61712572897954
BLEU3:  32.5762069658353
BLEU4:  19.894382214974662


In [38]:
val_actual, val_predicted = evaluate_model(old_experiment, val_df, report_vocab_length)

/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/vol/medic02/users/ag6516/python3env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

BLEU1:  67.27627032252504
BLEU2:  25.2024659697054
BLEU3:  15.010061038851074
BLEU4:  6.140149916104892


## Evaluate ROUGE scores on all train/val/test data

In [39]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                       max_n=4,
                       limit_length=True,
                       length_limit=100,
                       length_limit_type='words',
                       apply_avg='Avg',
                       apply_best='Best',
                       alpha=0.5, # Default F1_score
                       weight_factor=1.2,
                       stemming=True)

In [40]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)

In [ ]:
train_hypotheses = [' '.join(p) for p in train_predicted]
train_references = [' '.join(a) for a in train_actual]

scores = evaluator.get_scores(train_hypotheses, train_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

In [41]:
val_hypotheses = [' '.join(p) for p in val_predicted]
val_references = [' '.join(a) for a in val_actual]

scores = evaluator.get_scores(val_hypotheses, val_references)

for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print(prepare_results(results['p'], results['r'], results['f']))

	rouge-1:	P: 76.09	R: 71.92	F1: 72.54
	rouge-2:	P: 31.44	R: 28.17	F1: 28.46
	rouge-3:	P: 19.87	R: 18.53	F1: 18.45
	rouge-4:	P: 10.20	R:  9.70	F1:  9.54
	rouge-l:	P: 77.17	R: 73.51	F1: 74.24
	rouge-w:	P: 72.60	R: 58.96	F1: 62.89
